In [1]:
from crewai import Agent,Task,Crew,Process,LLM
from key import gooogle_api_key,serper_api_key
from crewai_tools import WebsiteSearchTool,ScrapeWebsiteTool,SerperDevTool

c:\Users\tjwin\anaconda3\envs\crewai_env\Lib\site-packages\pydantic\_internal\_config.py:295: PydanticDeprecatedSince20: Support for class-based `config` is deprecated, use ConfigDict instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  warnings.warn(DEPRECATION_MESSAGE, DeprecationWarning)
c:\Users\tjwin\anaconda3\envs\crewai_env\Lib\site-packages\pydantic\_internal\_generate_schema.py:502: UserWarning: <built-in function callable> is not a Python type (it may be an instance of an object), Pydantic will allow any object with no validation since we cannot even enforce that the input is an instance of the given type. To get rid of this error wrap the type with `pydantic.SkipValidation`.
  warn(
c:\Users\tjwin\anaconda3\envs\crewai_env\Lib\site-packages\crewai_tools\tools\scrapegraph_scrape_tool\scrapegraph_scrape_tool.py:34: PydanticDeprecatedSince20: Pydantic V1 style `@validator` validators are dep

In [2]:
import os
os.environ["GOOGLE_API_KEY"] = gooogle_api_key
os.environ["SERPER_API_KEY"] = serper_api_key

In [12]:
from crewai.tools import tool
from langchain_community.tools import DuckDuckGoSearchResults 
import json


@tool
def search_web_tool(query: str):
    """
    Searches the web and returns results.
    """
    search_tool = DuckDuckGoSearchResults(num_results=10, verbose=True)
    return search_tool.run(query)


# search_web_tool = WebsiteSearchTool(
#     config=dict(
#         llm=dict(
#             provider="ollama",
#             config=dict(
#                 model="llama2",
#             ),
#         ),
#         embedder=dict(
#             provider="google",
#             config=dict(
#                 model="models/embedding-001",
#                 task_type="retrieval_document",
#             ),
#         ),
#     )
# )


scraper = ScrapeWebsiteTool()
serper = SerperDevTool()

In [13]:
from crewai import LLM

llm = LLM(
    model="gemini/gemini-1.5-flash",
    temperature=0.7,
    #vertex_credentials=vertex_credentials_json
    api_key=gooogle_api_key
)

In [21]:
booking_agent = Agent(
    role = "Booking Specialist",
    goal = "Streamline a flights or trains (which ever will be the best)with in the budget. ",
    backstory = ("""You are skilled in finding the best deals and securing bookings quickly at cheapest."""),
    memory = True,
    allow_deligation = True,
    verbose = True,
    tools = [scraper,serper],
    llm = llm)

In [22]:
travel_planner_ag = Agent(
    role = "Travel Planner",
    goal = "Schedule the complete plan from day to night.",
    backstory = ("An expert in travel planning who knows the best destinations for every type of traveler."),
    tools = [scraper,serper],
    memory = True,
    allow_deligation = True,
    llm = llm ,verbose = True
)

In [ ]:
booking_task = Task(
    description=("""
    This task focuses on streamlining transportation (either flights or trains, whichever is the best) from {start_trip} on {start_date} to {destination} and return date will be next day from the stay. 
    Additionally, it includes finding suitable hotel accommodations in {destination} within the given budget of {Budget} for {people} people with a stay duration of {day} days.

    Key Considerations:
    - Prioritize the cheapest flight/train options while ensuring a balance between cost and convenience.
    - Find budget-friendly and well-rated hotels in {destination}.
    - Ensure all options are properly arranged in a structured format.
    - Provide direct booking links for flights, trains, and hotels.

    """),
    expected_output=f"""
    In markdown format: A structured report that categorizes available options under headings:
    - **Flights/Trains**: Cheapest available transportation options.
    - **Hotels**: Affordable and comfortable stay recommendations.
    
    Ensure that links to booking pages are included for all options.
    """,
    agent=booking_agent,
    tools=[scraper,serper],
)


In [31]:
travel_planner_task = Task(
    description = "streamline the places where user can visit including the main activity at {destination}.",
    agent = travel_planner_ag,
    expected_output = """
    In markdown format: A structured report that categorizes available options under headings:
    - **Flights/Trains**: Cheapest available transportation options with in the table having two rows.
    - **Hotels**: Affordable and comfortable stay recommendations with in the table having two rows.
    - **Schedule**: complete schedule from day to night with eating preferences with time stamps
    
    Ensure that links to booking pages are included for all options.
""",
    output_file="Plan.md"
)

In [32]:
crew_ = Crew(
    agents=[booking_agent,travel_planner_ag],
    tasks=[booking_task,travel_planner_task],
    process=Process.sequential,
    full_output=True,
    share_crew=False,
    verbose=True
)

2025-03-06 18:50:15,714 - 14284 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed


In [38]:
results = crew_.kickoff(inputs={
    "start_trip": "Chandigarh",
    "destination": "Bangalore",
    "Budget": 30000,
    "people": 2,
    "day": 2,
    "start_date": "2025-04-05"  # Use YYYY-MM-DD format
})


# Agent: Booking Specialist
## Task: 
    This task focuses on streamlining transportation (either flights or trains, whichever is the best) from Chandigarh to Bangalore. 
    Additionally, it includes finding suitable hotel accommodations in Bangalore within the given budget of 30000 for 2 people with a stay duration of 2 days.

    Key Considerations:
    - Prioritize the cheapest flight/train options while ensuring a balance between cost and convenience.
    - Find budget-friendly and well-rated hotels in Bangalore.
    - Ensure all options are properly arranged in a structured format.
    - Provide direct booking links for flights, trains, and hotels.

    


# Agent: Booking Specialist
## Thought: ```tool_code
Thought: I need to find the cheapest flights or trains from Delhi to Bhopal and hotels in Bhopal for 2 people with a 2-day stay within a budget of 20000.  I'll start by searching for flights and trains separately using Serper, then check hotel options.
## Using tool: Search 

In [39]:
from IPython.display import Markdown, display

# Path to your Markdown file


# Open and read the Markdown file
with open("Plan.md", "r", encoding="utf-8") as md_file:
    content = md_file.read()

# Render and display the Markdown content in the notebook
display(Markdown(content))

# Bangalore Trip Plan (Budget: Adjust as needed)

**Flights/Trains:**

| Transportation | Booking Link                                                                  | Estimated Price (per person, round trip) | Notes                                                                                                    |
|-----------------|------------------------------------------------------------------------------|------------------------------------------|---------------------------------------------------------------------------------------------------------|
| Flight          | [Multiple options available - check MakeMyTrip, Goibibo, Skyscanner, etc.] | ₹5000 - ₹10000                           | Prices vary greatly depending on the time of year and booking in advance is recommended for best prices. |
| Train           | [IRCTC](https://www.irctc.co.in/nget/train-search)                             | ₹2000 - ₹5000                            | Consider overnight trains to save on accommodation costs.  Check for availability well in advance.      |


**Hotels:**

| Hotel Type      | Booking Link                                                                          | Estimated Price (per night, double occupancy) | Notes                                                                                                                                                |
|-----------------|--------------------------------------------------------------------------------------|---------------------------------------------|-------------------------------------------------------------------------------------------------------------------------------------------------------|
| Budget-Friendly | [Multiple options available - check MakeMyTrip, Goibibo, Booking.com, OYO Rooms etc.] | ₹1000 - ₹3000                              | Look for hotels near the city center for easy access to attractions. Read reviews carefully before booking.                                                |
| Mid-Range       | [Multiple options available - check Expedia, Agoda, Hotels.com etc.]                | ₹3000 - ₹6000                              | Consider location and amenities when choosing a mid-range hotel.                                                                                       |


**Schedule:** (This is a sample schedule, adjust based on your interests and flight/train arrival times)

**Day 1:**

* **Morning (8:00 AM):** Arrive in Bangalore, check into hotel.
* **Late Morning (10:00 AM):** Visit Cubbon Park (Relaxing stroll, enjoy the greenery).
* **Lunch (1:00 PM):** South Indian Thali at MTR (Mavalli Tiffin Rooms) - a Bangalore institution.
* **Afternoon (2:30 PM):** Explore Vidhana Soudha (Impressive architecture, government buildings).
* **Late Afternoon (4:00 PM):** Visit Bangalore Palace (Beautiful architecture, historical significance).
* **Evening (6:00 PM):** Stroll through MG Road (Shopping, street food).
* **Dinner (8:00 PM):** North Indian cuisine at a restaurant on MG Road.
* **Night (9:30 PM):** Relax at the hotel.

**Day 2:**

* **Morning (8:00 AM):** Visit Lalbagh Botanical Garden (Beautiful gardens, diverse flora).
* **Late Morning (10:30 AM):** Visit Tipu Sultan's Summer Palace (Historical building, interesting exhibits).
* **Lunch (1:00 PM):**  Street food near the palace or a casual restaurant.
* **Afternoon (2:30 PM):** Explore Commercial Street (Shopping for clothes, souvenirs).
* **Late Afternoon (4:30 PM):** Visit ISKCON Temple (Spiritual experience, peaceful atmosphere).
* **Evening (6:00 PM):** Relax, perhaps enjoy a coffee at a local cafe.
* **Dinner (8:00 PM):** Choose a restaurant based on your preference (various cuisines available).
* **Night (9:30 PM):** Depart from Bangalore.


**Food Preferences:**  The schedule includes suggestions for South Indian and North Indian cuisine, but you can easily adapt it to your preferences. Bangalore offers a wide variety of culinary options.


**Note:** This is a suggested itinerary.  You can customize it based on your interests and time constraints.  Remember to check the opening and closing times of the attractions you plan to visit.  Transportation within Bangalore can be done using taxis, auto-rickshaws, or the metro.